# Сессия с полной производительностью

```python
# Создание SparkSession с настройками для максимальной производительности
spark = (SparkSession.builder
         # Имя приложения для идентификации в Spark UI и логах
         .appName("Full Power")
         
         # Подключение к Spark кластеру
         # spark-master:7077 - главный узел кластера, который управляет распределенными вычислениями
         .master("spark://spark-master:7077")
         
         # КОНФИГУРАЦИЯ ИСПОЛНИТЕЛЕЙ (EXECUTORS)
         # Executor - процесс, выполняющий задачи на рабочих узлах кластера
         
         # Количество исполнителей на рабочем узле
         # 2 исполнителя будут запущены на каждом worker node
         # Это позволяет лучше изолировать задачи и управлять памятью
         .config("spark.executor.instances", "2")  
         
         # Количество CPU ядер на каждый исполнитель
         # 10 ядер позволяют выполнять до 10 параллельных задач в каждом executor
         # Оптимально для узлов с большим количеством процессорных ядер
         .config("spark.executor.cores", "10")     
         
         # Объем оперативной памяти на каждый исполнитель
         # 16GB RAM для каждого executor процесса
         # Включает память для выполнения задач и хранения данных в памяти
         # Структура памяти executor:
         # - Execution Memory (шаффлинг, joins, агрегации)
         # - Storage Memory (кэшированные DataFrame/RDD)
         # - User Memory (пользовательские структуры данных)
         # - Reserved Memory (системные нужды Spark ~300MB)
         .config("spark.executor.memory", "16g")   
         
         # КОНФИГУРАЦИЯ ДРАЙВЕРА
         # Driver - главный процесс, координирующий выполнение приложения
         
         # Память драйвера - 4GB для:
         # - Хранения метаданных приложения
         # - Сбора результатов (collect())
         # - Broadcast переменных
         # - Управления задачами и планирования
         .config("spark.driver.memory", "4g")      
         
         # Создание или получение существующей сессии
         .getOrCreate()
        )
```

## Детальное объяснение архитектуры:

### **Распределение ресурсов:**
```
Worker Node (физическая/виртуальная машина)
├── Executor 1 (16GB RAM, 10 cores)
│   ├── Task Slot 1
│   ├── Task Slot 2
│   └── ... Task Slot 10
└── Executor 2 (16GB RAM, 10 cores)
    ├── Task Slot 1
    ├── Task Slot 2
    └── ... Task Slot 10
```

### **Производительность и масштабирование:**

1. **Параллелизм задач:**
   - Всего слотов для задач: `2 executors × 10 cores = 20 параллельных задач`
   - Это позволяет обрабатывать до 20 партиций данных одновременно

2. **Распределение памяти:**
   ```python
   # Примерная структура памяти executor (16GB):
   - Spark Memory: ~60% (9.6GB)
     * Storage Memory: кэшированные данные
     * Execution Memory: операции шаффлинга
   - User Memory: ~25% (4GB) - пользовательские структуры
   - Reserved: ~5% (0.8GB) - системные нужды
   ```

3. **Рекомендации для продакшена:**
   ```python
   # Дополнительные оптимизации для "Full Power":
   .config("spark.sql.adaptive.enabled", "true")          # Адаптивное выполнение запросов
   .config("spark.sql.adaptive.coalescePartitions.enabled", "true") # Объединение партиций
   .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") # Быстрая сериализация
   .config("spark.sql.shuffle.partitions", "200")         # Партиции для шаффлинга
   .config("spark.default.parallelism", "40")             # Параллелизм по умолчанию
   ```

### **Типичные сценарии использования:**
- **Обработка больших данных** (сотни GB - TB)
- **Сложные ETL-пайплайны** с множеством преобразований
- **Машинное обучение на больших датасетах**
- **Аналитика в реальном времени** с высокими требованиями к производительности

Такая конфигурация подходит для мощных продакшен-кластеров с значительными вычислительными ресурсами.

Отличная конфигурация! С вашим железом и текущим setup вы можете обрабатывать значительные объемы данных. Давайте разберем детально:

## **Текущие ресурсы кластера:**

### Вычислительная мощность:
```
Всего в кластере:
- 2 worker × 10 cores = 20 CPU ядер
- 2 worker × 18GB RAM = 36GB оперативной памяти
```

### Память для обработки данных:
```python
# Доступная память для данных (приблизительно):
Общая RAM: 36GB
- Системные нужды Spark: ~2GB
- Резерв: ~4GB
= Доступно для данных: ~30GB
```

## **Объемы данных для обработки:**

### **1. Оптимальные объемы (комфортная работа):**
- **В памяти:** 15-25 GB данных
- **На диске (с шаффлингом):** 50-100 GB данных
- **Ежедневная обработка:** 200-500 GB (с разбивкой на батчи)

### **2. Максимальные объемы (с оптимизацией):**
- **Единоразово в памяти:** до 30 GB
- **С промежуточной записью на диск:** 200-300 GB
- **Ежедневно:** 1-2 TB (с правильным партиционированием)

## **Рекомендации по оптимизации:**

### Конфигурация Spark для вашего железа:
```python
spark = (SparkSession.builder
    .appName("Optimized for i5-14600K")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")  
    .config("spark.executor.cores", "8")      # Оставляем 2 ядра на систему
    .config("spark.executor.memory", "14g")   # Оставляем 4GB на систему
    .config("spark.driver.memory", "8g")      # Увеличиваем для больших collect()
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128MB")
    .config("spark.sql.shuffle.partitions", "100")  # 8 cores × 2 workers × 6 = 96
    .config("spark.default.parallelism", "96")
    .config("spark.memory.fraction", "0.8")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)
```

## **Типичные сценарии обработки:**

### **1. Аналитические запросы:**
```python
# Мгновенная обработка: 5-10 GB
# Запросы до 1 минуты: 10-20 GB  
# Длительные ETL: 50-100 GB за запуск
```

### **2. Машинное обучение:**
```python
# Обучение моделей:
# - До 10M строк × 100 фич: комфортно
# - До 50M строк × 50 фич: с оптимизацией
# - Feature engineering: 20-30 GB наборов данных
```

### **3. Стриминг данных:**
```python
# Потоковая обработка:
# - До 50,000 событий/секунду
# - Микробатчи по 10-30 секунд
# - Окна агрегации: 1-60 минут
```

## **Мониторинг и ограничения:**

### Признаки нехватки ресурсов:
```python
# Тревожные сигналы:
- Frequent GC pauses (> 10% времени)
- Spill на диск (Disk spill bytes > 0)
- OOM errors
- Tasks taking > 5-10 minutes
```

### Практические примеры объемов:
```
✅ Легко:     Анализ 10GB CSV (~100M строк)
✅ Комфортно: ETL 50GB данных ежедневно  
⚠️ Возможно:  Обработка 150GB с оптимизацией
❌ Сложно:    Единовременная обработка >200GB
```

## **Рекомендации для вашего setup:**

1. **Используйте партиционирование** для больших datasets
2. **Кэшируйте часто используемые DataFrames**
3. **Настройте правильные типы данных** (меньше памяти)
4. **Используйте формат Parquet/ORC** вместо CSV/JSON
5. **Мониторьте Spark UI** на портах 8085-8087

Ваша система отлично подходит для серьезных data engineering задач и может обрабатывать гигабайты данных ежедневно!

# Сессия с MinIO

```python
# Создание SparkSession - основной точки входа для работы с Spark
spark = (SparkSession.builder
         .appName("MinIO Test")
         
         # Задаем URL мастер-ноды Spark кластера
         # spark://spark-master:7077 - означает, что Spark работает в кластерном режиме
         # spark-master - имя хоста мастер-ноды (обычно задается в Docker Compose или Kubernetes)
         # 7077 - стандартный порт для подключения драйверов к кластеру
         .master("spark://spark-master:7077") 
         
         # Указываем необходимые JAR-пакеты для работы с S3-совместимыми хранилищами
         # hadoop-aws:3.3.4 - библиотека Hadoop для работы с AWS S3
         # aws-java-sdk-bundle:1.12.262 - AWS Java SDK для клиентских операций
         # Spark автоматически скачает эти зависимости при запуске
         .config("spark.jars.packages", 
                 "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")
         
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
         
         # Указываем endpoint MinIO сервера
         # http://minio:9000 - MinIO работает по HTTP на порту 9000
         # 'minio' - имя сервиса в Docker-сети
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         
         # Включаем path-style доступ к бакетам (требуется для MinIO)
         # В отличие от virtual-hosted style, где бакет в домене: bucket.host.com
         # Path-style: host.com/bucket/path/to/file
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         
         # Указываем реализацию файловой системы для S3
         # S3AFileSystem - Hadoop файловая система для работы с S3-совместимыми хранилищами
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         
         # Отключаем SSL/TLS шифрование соединения
         # Так как мы используем HTTP (не HTTPS) для подключения к MinIO
         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
         .getOrCreate()
        )
```

## Ключевые моменты конфигурации:

### 1. **Архитектура подключения**
- Spark драйвер подключается к кластеру через мастер-ноду
- Executor'ы в кластере получат те же конфигурации S3/MinIO

### 2. **Работа с MinIO**
- MinIO эмулирует S3 API, поэтому используем S3A connector
- Path-style access обязателен для MinIO
- Отключен SSL т.к. используется HTTP

### 3. **Зависимости**
- `hadoop-aws` предоставляет S3A файловую систему  
- `aws-java-sdk` обеспечивает низкоуровневые S3 операции




# Запись CSV

Подробно разберем каждую опцию записи CSV в этом формате:

### Базовый синтаксис
```python
(df.write.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("quote", "\"")
    .option("escape", "\"")
    .option("encoding", "UTF-8")
    .mode("overwrite")
    .save("s3a://my-bucket/data/csv-output"))
```

---

### **1. `.format("csv")`**
**Назначение**: Указывает формат записи
```python
.format("csv")  # Запись в CSV формате
```
**Альтернативы**: `"parquet"`, `"json"`, `"orc"`, `"avro"`

---

### **2. `.option("header", "true")`**
**Назначение**: Включает запись заголовков столбцов
```python
.option("header", "true")   # ✅ Записать названия колонок
.option("header", "false")  # ❌ Без заголовков (по умолчанию)
```

**Пример данных:**
```
id,name,age
1,John,25
2,Jane,30
```

---

### **3. `.option("delimiter", ",")`**
**Назначение**: Задает разделитель полей
```python
.option("delimiter", ",")    # Стандартный CSV
.option("delimiter", ";")    # Европейский CSV
.option("delimiter", "|")    # Pipe-separated
.option("delimiter", "\t")   # TSV (табуляция)
```

**Результат с разными разделителями:**
```csv
# delimiter = ","
1,John,25

# delimiter = "|"
1|John|25
```

---

### **4. `.option("quote", "\"")`**
**Назначение**: Символ для обрамления полей
```python
.option("quote", "\"")    # Двойные кавычки (стандарт)
.option("quote", "'")     # Одинарные кавычки
.option("quote", "")      # Без quoting (опасно!)
```

**Когда используется:**
```csv
# Без quote
1,John Doe,25"  # ❌ Проблема с кавычкой в данных

# С quote
1,"John Doe","25""",Engineer  # ✅ Экранирование работает
```

---

### **5. `.option("escape", "\"")`**
**Назначение**: Символ экранирования специальных символов
```python
.option("escape", "\"")    # Экранирование кавычкой
.option("escape", "\\")    # Экранирование обратным слэшем
```

**Пример экранирования:**
```csv
# Данные: He said "Hello"
"He said ""Hello"""     # escape = "\""
"He said \"Hello\""     # escape = "\\"
```

---

### **6. `.option("encoding", "UTF-8")`**
**Назначение**: Кодировка файла
```python
.option("encoding", "UTF-8")      # Unicode (рекомендуется)
.option("encoding", "ISO-8859-1") # Latin-1
.option("encoding", "CP1251")     # Windows Cyrillic
```

---

### **7. `.mode("overwrite")`**
**Назначение**: Режим записи при существующем файле
```python
.mode("overwrite")   # 📝 Перезаписать существующие данные
.mode("append")      # ➕ Добавить к существующим
.mode("ignore")      # ⏭️ Пропустить если файл существует
.mode("error")       # ❌ Ошибка если файл существует (по умолчанию)
```

---

### **Дополнительные важные опции**

### **Управление NULL значениями**
```python
.option("nullValue", "NULL")     # Заменяет null на "NULL"
.option("nullValue", "")         # Заменяет null на пустую строку
.option("nullValue", "\\N")      # Как в MySQL
```

### **Форматы дат и времени**
```python
.option("dateFormat", "yyyy-MM-dd")          # 2024-01-15
.option("timestampFormat", "yyyy-MM-dd HH:mm:ss")  # 2024-01-15 14:30:00
```

### **Сжатие**
```python
.option("compression", "gzip")    # .csv.gz
.option("compression", "none")    # Без сжатия
```

---

## **Полный пример с всеми опциями**
```python
(df.write.format("csv")
    .option("header", "true")              # Заголовки
    .option("delimiter", ",")              # Разделитель
    .option("quote", "\"")                 # Символ quoting
    .option("escape", "\"")                # Символ экранирования
    .option("encoding", "UTF-8")           # Кодировка
    .option("nullValue", "NULL")           # Замена null
    .option("dateFormat", "yyyy-MM-dd")    # Формат дат
    .option("compression", "gzip")         # Сжатие
    .mode("overwrite")                     # Режим записи
    .save("s3a://my-bucket/data/output"))  # Путь назначения
```

---

## **Результат записи**
Для DataFrame:
```python
+---+----------+---+
| id|      name|age|
+---+----------+---+
|  1|  John Doe| 25|
|  2|Jane Smith| 30|
|  3|      NULL| 35|
+---+----------+---+
```

Будет записан CSV:
```csv
id,name,age
1,"John Doe",25
2,"Jane Smith",30
3,NULL,35
```

---

# Запись Parquet


```python
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .mode("overwrite")
    .save("s3a://my-bucket/data/parquet-output"))
```

---

## **1. `.format("parquet")`**
**Назначение**: Указывает формат Parquet
```python
.format("parquet")  # Запись в колоночном формате Parquet
```

---

## **2. `.option("compression", "snappy")`**
**Назначение**: Алгоритм сжатия данных
```python
.option("compression", "snappy")    # 🚀 Быстрое сжатие (рекомендуется)
.option("compression", "gzip")      # 📦 Высокое сжатие (медленнее)
.option("compression", "lzo")       # ⚡ Очень быстрое (меньше сжатие)
.option("compression", "none")      # 🔓 Без сжатия
.option("compression", "zstd")      # 🎯 Баланс скорости/сжатия (новое)
```

**Сравнение:**
- **Snappy**: Скорость + разумное сжатие
- **Gzip**: Медленнее, но лучшее сжатие
- **None**: Для часто изменяемых данных

---

## **3. `.option("parquet.block.size", 134217728)`**
**Назначение**: Размер блока данных в байтах
```python
.option("parquet.block.size", 134217728)  # 128 MB (по умолчанию)
.option("parquet.block.size", 268435456)  # 256 MB (для больших данных)
.option("parquet.block.size", 67108864)   # 64 MB (для маленьких файлов)
```

**Зачем нужно:**
- Блок - минимальная единица чтения
- Большие блоки → лучшее сжатие
- Меньшие блокы → лучше параллелизм

---

## **4. `.option("parquet.page.size", 1048576)`**
**Назначение**: Размер страницы в байтах
```python
.option("parquet.page.size", 1048576)     # 1 MB (по умолчанию)
.option("parquet.page.size", 524288)      # 512 KB
.option("parquet.page.size", 2097152)     # 2 MB
```

**Зачем нужно:**
- Страница - минимальная единица доступа
- Меньший размер → точечное чтение
- Больший размер → лучшее сжатие

---

## **5. `.option("parquet.dictionary.enabled", "true")`**
**Назначение**: Включить словарное кодирование
```python
.option("parquet.dictionary.enabled", "true")   # ✅ Включить (рекомендуется)
.option("parquet.dictionary.enabled", "false")  # ❌ Выключить
```

**Преимущества:**
- Эффективно для повторяющихся значений
- Уменьшает размер данных
- Особенно полезно для строковых колонок

---

## **6. `.option("parquet.bloom.filter.enabled", "true")`**
**Назначение**: Включить Bloom filter для быстрого поиска
```python
.option("parquet.bloom.filter.enabled", "true")    # ✅ Включить
.option("parquet.bloom.filter.enabled", "false")   # ❌ Выключить
```

**Зачем нужно:**
- Ускоряет предикатный pushdown
- Эффективно для поиска по конкретным значениям
- Особенно полезно для колонок с высоким кардиналитетом

---

## **7. `.mode("overwrite")`**
**Назначение**: Режим записи
```python
.mode("overwrite")   # 📝 Перезаписать существующие данные
.mode("append")      # ➕ Добавить к существующим
.mode("ignore")      # ⏭️ Пропустить если существует
.mode("error")       # ❌ Ошибка если существует
```

---

## **Дополнительные важные опции**

### **Настройка производительности**
```python
.option("parquet.writer.version", "v2")           # Версия формата
.option("parquet.enable.dictionary", "true")      # Словарное кодирование
.option("parquet.dictionary.page.size", 1048576)  # Размер словарной страницы
```

### **Схема данных**
```python
.option("mergeSchema", "true")                    # Объединять схемы при append
.option("parquet.strings.signed-min-max.enabled", "true")  # Для строковых статистик
```

### **Партиционирование**
```python
.partitionBy("year", "month")  # 🗂️ Партиционирование по колонкам
```

---

## **Полный пример с партиционированием**

```python
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .option("parquet.writer.version", "v2")
    .option("mergeSchema", "true")
    .partitionBy("year", "month")           # Партиционирование
    .mode("overwrite")
    .save("s3a://my-bucket/data/partitioned-parquet/"))
```

---

## **Структура результата в MinIO**
```
s3a://my-bucket/data/parquet-output/
├── part-00000-xxx.snappy.parquet
├── part-00001-xxx.snappy.parquet
└── _SUCCESS

s3a://my-bucket/data/partitioned-parquet/
├── year=2024/
│   ├── month=01/
│   │   └── part-00000-xxx.snappy.parquet
│   └── month=02/
│       └── part-00000-xxx.snappy.parquet
└── year=2023/
    └── month=12/
        └── part-00000-xxx.snappy.parquet
```

---

## **Преимущества Parquet в MinIO**
- **✅ Колоночный формат** - эффективные запросы
- **✅ Сжатие** - экономия хранилища
- **✅ Предикатный pushdown** - фильтрация на чтении
- **✅ Schema evolution** - эволюция схемы данных
- **✅ Совместимость** - работа с различными инструментами